## Data Source specific clean-up of Data
# Only Needs to be done Once

In [1]:

# Clean Data and place into a symbols_dict

import glob
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import vectorbtpro as vbt
from logger_tt import setup_logging
from tsfresh.utilities.dataframe_functions import make_forecasting_frame  # noqa

from Modules._Data_Manager import Data_Manager

# import dask.dataframe as dd

setup_logging(full_context=1)

In [2]:
from pandas_datareader import DataReader

symbols_list = ['GE', 'IBM','MSFT']
d = {}
for ticker in symbols_list:
    d[ticker] = DataReader(ticker, "yahoo", '1950-01-01')

d

{'GE':                  High        Low       Open      Close     Volume  Adj Close
 Date                                                                        
 1962-01-02   6.109776   5.949519   6.009615   5.989583   269568.0   1.008423
 1962-01-03   5.959535   5.909455   5.959535   5.929487   184704.0   0.998305
 1962-01-04   5.979567   5.809295   5.929487   5.859375   229632.0   0.986501
 1962-01-05   5.869391   5.608974   5.859375   5.709135   340704.0   0.961206
 1962-01-08   5.709135   5.528846   5.709135   5.699119   386880.0   0.959520
 ...               ...        ...        ...        ...        ...        ...
 2022-10-10  65.620003  64.059998  65.050003  64.980003  3591200.0  64.980003
 2022-10-11  65.260002  63.080002  64.489998  64.000000  5295900.0  64.000000
 2022-10-12  65.419998  63.060001  63.689999  64.739998  5356900.0  64.739998
 2022-10-13  68.360001  63.189999  63.689999  67.940002  7984500.0  67.940002
 2022-10-14  68.779999  67.190002  68.300003  67.260002  3

In [3]:
from Models.Market_Wide.SP500_Predictor import fetch_index_asset_prices, fetch_alternative_data

"""Load Raw Data"""

# Duka allows for download multiple assets at once however due to the limit imposed by the data source it tends
# to cause perfoimary issues
# ! duka EURUSD  -s 2020-09-29  --header && duka USDJPY -s 2020-09-29  --header &&
#       duka GBPUSD  -s 2020-09-29  --header && duka AUDUSD  -s 2020-09-29  --header &&
#       duka USDCAD -s 2020-09-29  --header && duka USDCNY  -s 2020-09-29  --header &&
#       duka USDCHF  -s 2020-09-29  --header && duka EURGBP  -s 2020-09-29  --header &&
#       duka USDKRW  -s 2020-09-29  --header

# file_names = [path.split("/")[-1] for path in
#               glob.glob("/home/ruben/PycharmProjects/mini_Genie_ML/Datas/Forex_Tick_Data/*.csv")]
#
# asset_data_obj = Data_Manager().fetch_data(data_file_names=file_names,
#                                            data_file_dirs=[
#                                                "/home/ruben/PycharmProjects/mini_Genie_ML/Datas/Forex_Tick_Data/"],
#                                            )
asset_data_obj = fetch_index_asset_prices(days_to_fetch=10, last_day=datetime.today() - timedelta(days=1)
                                          , index_name="SP500", read_from_file=False)
alternative_data_dir = "/home/ruben/PycharmProjects/mini_Genie_ML/Datas/Alternative_Data"

# Label data
AD_df_obj = fetch_alternative_data(alternative_data_dir)

# Print all columns
pd.set_option('display.max_columns', None)
asset_data_obj.save("sp500_data_obj_data")
AD_df_obj.save("AD_df_obj_data")



Fetching 504 assets from 2022-10-03 14:23:08.302395 to 2022-10-13 14:23:08.302395 found in the S&P500
following the added and removed tickers from the index over time
Fetching A data
                  High         Low        Open       Close     Volume  \
Date                                                                    
2022-10-03  127.300003  121.680000  122.730003  126.379997  1343200.0   
2022-10-04  131.559998  128.570007  128.789993  131.410004  1647800.0   
2022-10-05  133.529999  129.500000  129.990005  132.639999  1567700.0   
2022-10-06  133.820007  131.600006  132.300003  132.179993  1218400.0   
2022-10-07  130.929993  126.959999  130.449997  127.440002  1154700.0   

             Adj Close  
Date                    
2022-10-03  126.379997  
2022-10-04  131.410004  
2022-10-05  132.639999  
2022-10-06  132.179993  
2022-10-07  127.440002  
A_High
A_Low
A_Open
A_Close
A_Volume
A_Adj Close
Fetching AAL data
             High    Low   Open  Close    Volume  Adj Close
Dat


KeyboardInterrupt



In [ ]:
asset_data_obj

In [ ]:
AD_df_obj


In [ ]:
"""STEP __N__"""
"""Load Combined Data Pickle"""
symbols_dict_obj = vbt.Data.load("forex_tick_data").data

In [ ]:
import Genie as genie

_preprocessed_data_dict = dict()

for asset in symbols_dict_obj.keys():
    # smalled_index = symbols_dict_obj[asset].first_valid_index()
    # largest_index = symbols_dict_obj[asset].last_valid_index()
    #
    # _preprocessed_data_dict[asset[:6]] = symbols_dict_obj[asset][smalled_index:largest_index].copy()
    _preprocessed_data_dict[asset[:6]] = symbols_dict_obj[asset].copy()
    asset = asset[:6]

    _preprocessed_data_dict[asset] = _preprocessed_data_dict[asset].dropna()
    # spread = abs(
    #     preprocessed_data_dict[asset]["ask"] - preprocessed_data_dict[asset]["bid"])

    threshold =
    (_preprocessed_data_dict[asset]['ask'] * _preprocessed_data_dict[asset]['ask_volume']).vbt.resample_apply('1 min',
                                                                                                              vbt.nb.min_reduce_nb).dropna().describe()[
        '75%'].mean()
    print(f'{threshold = }')
    _preprocessed_data_dict[asset] = genie.ml.data_structures.get_dollar_bars(
        _preprocessed_data_dict[asset].reset_index()[["time", "ask", "ask_volume"]],
        threshold=threshold,
        batch_size=1000000, verbose=True)

    # Constant Run Bars
    # _preprocessed_data_dict[asset] = genie.ml.data_structures.get_const_dollar_run_bars(
    #     _preprocessed_data_dict[asset].reset_index()[["time", "ask", "ask_volume"]], num_prev_bars=3,
    #     exp_num_ticks_init=100,
    #     expected_imbalance_window=100)[0].set_index("date_time")

    _preprocessed_data_dict[asset].to_pickle(
        f'/home/ruben/PycharmProjects/mini_Genie_ML/Datas/Forex_Tick_Data/{asset}_dollar_run_bars.pickle')

    # print(preprocessed_data_dict[asset])
    print(_preprocessed_data_dict[asset])



In [ ]:
_preprocessed_data_dict.keys()
preprocessed_data_dict = _preprocessed_data_dict
preprocessed_data_dict

In [ ]:
file_paths = glob.glob("/home/ruben/PycharmProjects/mini_Genie_ML/Datas/Forex_Tick_Data/*_dollar_run_bars.pickle")
preprocessed_data_dict = dict()
for path in file_paths:
    preprocessed_data_dict[path.split("/")[-1][:6]] = pd.read_pickle(path).set_index('date_time')

##Step 1 - A

In [ ]:
'''Compute y'''
from multiprocessing import cpu_count
from __utils import genie_strategy_wrapper
import numpy as np

parametrized_genie_strategy_wrapper = vbt.parameterized(genie_strategy_wrapper,
                                                        # merge_func="concat",
                                                        # n_chunks=np.floor(param_combinations.shape[0]/4).astype(int),
                                                        # n_chunks=np.floor(param_combinations.shape[0]/4).astype(int),
                                                        chunk_len='auto',
                                                        engine='ray',
                                                        show_progress=True,
                                                        init_kwargs={
                                                            # 'address': 'auto',
                                                            'num_cpus': cpu_count() - 2,
                                                            # 'n_chunks':"auto",
                                                            # 'memory': 100 * 10 ** 9,
                                                            # 'object_store_memory': 100 * 10 ** 9,
                                                        },
                                                        )

assets_data_list = [preprocessed_data_dict[asset_ohlc] for asset_ohlc in preprocessed_data_dict.keys()]
result = parametrized_genie_strategy_wrapper(
    asset_ohlc=vbt.Param(
        assets_data_list
        # , name='symbols'
    ),
    threads_per_worker=np.floor((cpu_count() - 2) / len(assets_data_list)).astype(int),

)
# pd.set_option('display.max_columns', None)

first_index = preprocessed_data_dict[list(preprocessed_data_dict.keys())[0]].index[0]
last_index = preprocessed_data_dict[list(preprocessed_data_dict.keys())[0]].index[-1]

for index, asset_ohlc in enumerate(preprocessed_data_dict.keys()):
    # print(result[index])
    preprocessed_data_dict[asset_ohlc]["target"] = result[index]
    preprocessed_data_dict[asset_ohlc] = preprocessed_data_dict[asset_ohlc].fillna(
        0)  #.fillna(method="ffill")  #.dropna(0)
    # first_index = preprocessed_data_dict[asset_ohlc].index[0] if first_index < preprocessed_data_dict[asset_ohlc].index[
    #     0] else first_index
    # last_index = preprocessed_data_dict[asset_ohlc].index[-1] if last_index > preprocessed_data_dict[asset_ohlc].index[
    #     -1] else last_index

    # print(f"{(preped_forex_data[asset_ohlc]['target'] == 1).sum() = }")
    # print(f"{(preped_forex_data[asset_ohlc]['target'] == -1).sum() = }")
    # print(f"{(preped_forex_data[asset_ohlc]['target'] == 0).sum() = }")

    print(preprocessed_data_dict[asset_ohlc])



In [ ]:
preprocessed_data_dict[asset_ohlc]['target'].sum()

In [ ]:
def add_prefix_to_df_columns(df, prefix):
    df.columns = [prefix + "_" + col for col in df.columns]
    return df


for asset_name in preprocessed_data_dict.keys():
    # preprocessed_data_dict[asset_name] = preprocessed_data_dict[asset_name].loc[first_index:last_index]
    preprocessed_data_dict[asset_name] = add_prefix_to_df_columns(preprocessed_data_dict[asset_name], asset_name)

    print(preprocessed_data_dict[asset_name].head())
preprocessed_data_dict
# e.g.

In [ ]:
df = pd.concat(preprocessed_data_dict, axis=1).reset_index()

cols = list(df.columns.droplevel(0))
cols[0] = 'date_time'
df.columns = cols
df = df.ffill()
df = df.dropna()

In [ ]:
df = df.set_index("date_time")
df = df.tz_convert(None)


In [ ]:
df.to_csv('mmt_triplebarrier_h2o_test_dataset.csv')


In [ ]:
forex_data = pd.read_csv('mmt_triplebarrier_h2o_test_dataset.csv')
forex_data

In [ ]:
forex_data = df.copy()

In [ ]:
import h2o
from h2o.automl import H2OAutoML

In [ ]:
h2o.init(nthreads=4)


In [ ]:

# Identify predictors and response
x = list(forex_data.columns)
y = "EURUSD_target"
x.remove(y)

# pandas dataframe to h2o dataframe
h2o_forex_data = h2o.H2OFrame(forex_data)

# Split data into train and test sets,and  validation
train, valid, test = h2o_forex_data.split_frame([0.8, 0.1], seed=1234)

# For binary classification, response should be a factor
train[y] = train[y].asfactor()
valid[y] = valid[y].asfactor()
test[y] = test[y].asfactor()


In [ ]:
h2o.cluster().shutdown()


In [ ]:
# Run AutoML for 20 base models
aml = H2OAutoML(max_models=20, seed=1)
aml.train(x=x, y=y, training_frame=train)

# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

## Step 1 - B
Compute Alphas on Prices and add them to their respective dataframes in the symbols dict

In [ ]:
"""Compute alphas"""
# wqa_indicator = vbt.wqa101(53)
# value = wqa_indicator.run(open=asset_ohlc['open'], high=asset_ohlc['high'], low=asset_ohlc['low'],
#                           close=asset_ohlc['close'], volume=asset_ohlc['volume'])
# value.out

## Step 2
Transform symbols_dict into a flat_dataframe for tsfresh use and a performace boost

In [ ]:
"""Prep for TSFRESH"""
# forex_data_dict = preprocessed_data_dict.copy()
# forex_concated = pd.concat(forex_data_dict, axis=0)  # ["symbols"].extend(list(forex_data_dict.keys()))
# pd.melt(forex_concated)
# forex_concated["id"] = forex_concated.index
forex_concated = forex_data.reset_index()
# Change level_0 column to symbol
forex_concated = forex_concated.rename(columns={'index': 'id'})

# forex_concated = forex_concated.drop(columns=["level_0"])
forex_concated.to_pickle('flat_forex_dataframe.pickle')
print(forex_concated)

## Step 3
1. Create a rolling time series (n windows)
2. Compute the features (In this case only those that can be efficiently computed) "X".
3. Prepare y
4. Select only those features based on their p-value significance in the classification or regression target (y)
5.

In [ ]:
"""TSFRESH Processes"""
import vectorbtpro as vbt
import numpy as np
import pandas as pd

tsfresh_ready_data = pd.read_pickle("flat_forex_dataframe.pickle")
tsfresh_ready_data = tsfresh_ready_data.dropna()

# %%
from tsfresh.utilities.dataframe_functions import roll_time_series

_tsfresh_ready_data_rolled = roll_time_series(tsfresh_ready_data,
                                              column_id="id",
                                              column_sort="date_time",
                                              max_timeshift=1, min_timeshift=1, rolling_direction=1)
# %%
_tsfresh_ready_data_rolled

In [ ]:
_tsfresh_ready_data_rolled.to_pickle('tsfresh_ready_data_rolled.pickle')

In [ ]:
import pandas as pd

tsfresh_ready_data_rolled = pd.read_pickle("tsfresh_ready_data_rolled.pickle")
# tsfresh_ready_data_rolled

In [ ]:
target = tsfresh_ready_data_rolled.pop('target')
target = target.astype(int)
target

In [ ]:
# """TSFRESH Processes"""
# import vectorbtpro as vbt
# import numpy as np
# import pandas as pd
# from tsfresh.feature_extraction import extract_features
#
# tsfresh_ready_data = pd.read_pickle("flat_forex_dataframe.pickle")
# tsfresh_ready_data = tsfresh_ready_data.dropna()
#
# # %%
# from tsfresh.utilities.dataframe_functions import roll_time_series
#
# tsfresh_ready_data_rolled = roll_time_series(tsfresh_ready_data, column_id="id",
#                                              # column_sort="date_time",
#                                              max_timeshift=200, min_timeshift=200, rolling_direction=1)
# # %%

# %%
from tsfresh.utilities.distribution import ClusterDaskDistributor, LocalDaskDistributor  # noqa: F401
from tsfresh.utilities.distribution import MultiprocessingDistributor  # noqa: F401
from tsfresh.feature_extraction.settings import ComprehensiveFCParameters, EfficientFCParameters  # noqa: F401
from tsfresh.feature_extraction import extract_features

# settings = EfficientFCParameters()
settings = {
    # "length": None,
    "large_standard_deviation": [{"r": 0.05}, {"r": 0.1}]
}

# We construct a Distributor that will spawn the calculations
# over four threads on the local machine
# Distributor = MultiprocessingDistributor(n_workers=14,
#                                          disable_progressbar=False,
#                                          progressbar_title="Feature Extraction")

# Distributor = ClusterDaskDistributor(address='localhost:8786')


# Distributor = LocalDaskDistributor(n_workers=20)
# X_tsfresh contains the extracted tsfresh features
# just to pass the Distributor object to
# the feature extraction, along with the other parameters


X_tsfresh = extract_features(
    timeseries_container=tsfresh_ready_data_rolled,
    column_id='id',
    column_sort='date_time',
    # distributor=Distributor,
    n_jobs=16,
    default_fc_parameters=settings,
)
# %%
X_tsfresh


In [ ]:
X_tsfresh.to_pickle('X_tsfresh.pickle')

In [ ]:
target.shape

In [ ]:
from tsfresh import select_features

# which are now filtered to only contain relevant features
X_tsfresh_filtered = select_features(X_tsfresh, target)
X_tsfresh_filtered
# # # we can easily construct the corresponding settings object
# kind_to_fc_parameters = feature_extraction.settings.from_columns(X_tsfresh_filtered)
# %%

In [ ]:
# from Modules.Utils import mltidx_df_to_dict
#
# preped_forex_data_dict = mltidx_df_to_dict(preped_forex_data)

In [ ]:
# # which are now filtered to only contain relevant features
# X_tsfresh_filtered = tsf.some_feature_selection(X_tsfresh, y, ....)
#
# # we can easily construct the corresponding settings object
# kind_to_fc_parameters = tsf.feature_extraction.settings.from_columns(X_tsfresh_filtered)

In [ ]:
from multiprocessing import cpu_count
from __utils import genie_strategy_wrapper

"""Pass Through a Strategy"""
clean_forex_data_dict = dict()
for asset_ohlc in tuple(forex_data_dict.data.keys()):
    clean_forex_data_dict[asset_ohlc] = forex_data_dict[asset_ohlc].set_index("datetime")


In [ ]:



parametrized_genie_strategy_wrapper = vbt.parameterized(genie_strategy_wrapper,
                                                        # merge_func="concat",
                                                        # n_chunks=np.floor(param_combinations.shape[0]/4).astype(int),
                                                        # n_chunks=np.floor(param_combinations.shape[0]/4).astype(int),
                                                        chunk_len='auto',
                                                        engine='ray',
                                                        show_progress=True,
                                                        init_kwargs={
                                                            # 'address': 'auto',
                                                            'num_cpus': cpu_count() - 2,
                                                            # 'n_chunks':"auto",
                                                            # 'memory': 100 * 10 ** 9,
                                                            # 'object_store_memory': 100 * 10 ** 9,
                                                        },
                                                        )

assets_list = [clean_forex_data_dict[asset_ohlc] for asset_ohlc in clean_forex_data_dict.keys()]
result = parametrized_genie_strategy_wrapper(
    asset_ohlc=vbt.Param(
        assets_list
        # , name='symbols'
    ),
    threads_per_worker=np.floor((cpu_count() - 2) / len(assets_list)).astype(int),

)


In [ ]:
"""Alternative Data ... """

AD_dobj = Data_Manager().fetch_data(
    data_file_names="AD_df.csv",
    data_file_dirs=
    ["/home/ruben/PycharmProjects/mini_Genie_ML/Datas/Forex_Tick_Data", ".",
     "/home/ruben/PycharmProjects/mini_Genie_ML/Datas/Alternative_Data"],
)
AD_dobj.data